In [12]:
################################################################################################
#                                   最优订货量决策例子
#                                    （2021-02-27）
################################################################################################

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import math
import itertools

from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# 任务一：定义并检查总成本计算函数

## 1. 定义成本计算函数 inventory_cost_df

In [13]:
#-----------------------------------------------------------------------------------------------#
# inventory_cost_df：给定订货策略 Q_list，成本结果函数
#-----------------------------------------------------------------------------------------------#
def inventory_cost_df(days, P_list, D_list, Q_list, S=240, C=3, IC=10, fast=False):
    
    ### params ###
    # 单位订货固定成本（shipment cost）：S = 240
    # 单位产品日均存货成本：C = 3
    # 单位产品日均缺货成本：IC = 10

    # 准备空列表
    day_i_list = [i for i in range(days)] # 总天数
    Stock_list = [None for i in range(days)]
    Ins_list = [None for i in range(days)]
    Cost_pq_list = [None for i in range(days)]
    Cost_ship_list = [None for i in range(days)]
    Cost_stock_list = [None for i in range(days)]
    Cost_ins_list = [None for i in range(days)]
    Cost_list = [None for i in range(days)]
    
    ### 按日处理 ###
    for day_i in range(days):
        
        # 昨日库存：
        if day_i == 0:
            l_stock = 0
        else:
            l_stock = Stock_list[day_i-1]
        # 今日价格：
        p = P_list[day_i]
        # 今日订货量：
        q = Q_list[day_i]
        # 今日需求量：
        d = D_list[day_i]
        # 更新今日库存 = 昨日库存 + 今日购进 - 今日使用量
        stock = l_stock + q - d # 今日结束时的库存（负数表示缺货）
        Stock_list[day_i] = stock # 记录
        # 今日缺货量：
        if Stock_list[day_i] < 0:
            ins = abs(Stock_list[day_i]) # 缺货量（正数）
        else:
            ins = 0
        Ins_list[day_i] = ins # 记录
        
        ### 今日成本计算 ###
        # 购进产品总价值
        cost_pq = q * p 
        # 运输成本 ，如果进货（q > 0），则付出运输成本 S，否则为 0
        if q > 0:
            cost_ship = S 
        else:
            cost_ship = 0
        # 存储成本，假设日内需求均匀发生，带来半数的等效库存成本
        cost_stock = C * (max(stock, 0) + min(d, max(q + l_stock, 0))/2) 
        # 缺货成本 = IC * 缺货量（正数）
        cost_ins = IC * ins 
        # 总成本
        cost = cost_pq + cost_ship + cost_stock + cost_ins 
        
        # 修改相应 list 中该日的数值
        Cost_pq_list[day_i] = cost_pq
        Cost_ship_list[day_i] = cost_ship
        Cost_stock_list[day_i] = cost_stock
        Cost_ins_list[day_i] = cost_ins
        Cost_list[day_i] = cost
        
    ### 存入 pandas df ###
    if fast == True:
        data = Cost_list
        df = pd.DataFrame(data=data, columns=['总成本'])
        df['累计总成本'] = df['总成本'].cumsum()
    else:
        data = zip(day_i_list, P_list, D_list, Q_list, Stock_list, Ins_list, Cost_list, Cost_pq_list, Cost_ship_list, Cost_stock_list, Cost_ins_list)
        df = pd.DataFrame(data=data, columns=['日期', '价格', '需求量', '订货量', '库存量', '缺货量', '总成本', '购进货物价值', '运输成本', '库存成本', '缺货成本'])
        df['累计需求量'] = df['需求量'].cumsum()
        df['累计订货量'] = df['订货量'].cumsum()
        df['累计总成本'] = df['总成本'].cumsum()
        df['累计运输成本'] = df['运输成本'].cumsum()
        df['累计库存成本'] = df['库存成本'].cumsum()
        df['累计缺货成本'] = df['缺货成本'].cumsum()
    
    #### return ###
    return df


## 2. 用 EOQ 公式验证成本函数的正确性

### 2.1 根据 EOQ 公式，计算 Economic Order Quantity 的 Q_list

In [14]:
#-----------------------------------------------------------------------------------------------#
# 用 EOQ 公式，计算 Economic Order Quantity (EOQ) 的 Q_list
#-----------------------------------------------------------------------------------------------#
### params ###
# 总天数：
days = 60
day_i_list = [i for i in range(days)]
# 日均订货量（demand）：
D = 10
# 产品单位价格：
P = 20
# 单位订货固定成本（shipment cost）：
S = 240.0
# 单位产品日均存货成本：
C = 3.0
# 单位产品日均缺货成本：
IC = 10.0
    
### Q_list_EOQ ###
# 最优每次订货量（quantity）：Q = sqrt(2 * D * S / C)
Q = (2 * D * S / C) ** 0.5
print(f'EOQ公式 最优每次订货量：{Q}')
# 最优订货次数：N = D * days / Q
N = D * days / Q
print(f'EOQ公式 最优订货次数：{N}')
# 订货周期天数：T = 1 / N * days
T = 1 / N * days
print(f'EOQ公式 订货周期天数：{T}')
# 订货计划 Q_list_EOQ
Q_list_EOQ = []
for day_i in range(days):
    # 如果到了订货周期
    if day_i % T == 0:
        Q_list_EOQ.append(Q)
    else:
        Q_list_EOQ.append(0)

print(f'EOQ公式 订货计划：{Q_list_EOQ}')   

EOQ公式 最优每次订货量：40.0
EOQ公式 最优订货次数：15.0
EOQ公式 订货周期天数：4.0
EOQ公式 订货计划：[40.0, 0, 0, 0, 40.0, 0, 0, 0, 40.0, 0, 0, 0, 40.0, 0, 0, 0, 40.0, 0, 0, 0, 40.0, 0, 0, 0, 40.0, 0, 0, 0, 40.0, 0, 0, 0, 40.0, 0, 0, 0, 40.0, 0, 0, 0, 40.0, 0, 0, 0, 40.0, 0, 0, 0, 40.0, 0, 0, 0, 40.0, 0, 0, 0, 40.0, 0, 0, 0]


### 2.2 根据 EOQ 公式，计算 Economic Order Quantity 的总成本

In [15]:
### 用 EOQ 公式，计算 Economic Order Quantity (EOQ) 的成本

# EOQ 模型总固定订购成本（运输成本）：S * N = S * D * days / Q
print(f'EOQ公式 总运输成本：{S * D * days / Q}')
# EOQ 模型总库存成本（储藏/存放成本）：(Q / 2) * C * days
print(f'EOQ公式 总库存成本：{(Q / 2) * C * days}')
# EOQ 模型总成本：
print(f'EOQ公式 总成本：{S * D * days / Q + (Q / 2) * C * days}')


EOQ公式 总运输成本：3600.0
EOQ公式 总库存成本：3600.0
EOQ公式 总成本：7200.0


### 2.3 用自定义函数计算 EOQ 模型的成本，检查是否与公式结果一致

In [16]:
### 用自定义函数计算 EOQ 模型的成本 ###
days = 60
P_list = [P for i in range(days)]
D_list = [D for i in range(days)]
# inventory_cost_df(days, P_list, D_list, Q_list_EOQ, fast=True)
df = inventory_cost_df(days, P_list, D_list, Q_list_EOQ)
# df[['累计需求量', '累计订货量', '累计总成本', '累计运输成本', '累计库存成本', '累计缺货成本']].iloc[-1]
df

,日期,价格,需求量,订货量,库存量,缺货量,总成本,购进货物价值,运输成本,库存成本,缺货成本,累计需求量,累计订货量,累计总成本,累计运输成本,累计库存成本,累计缺货成本
0,0,20,10,40.0,30.0,0,1145.0,800.0,240,105.0,0,10,40.0,1145.0,240,105.0,0
1,1,20,10,0.0,20.0,0,75.0,0.0,0,75.0,0,20,40.0,1220.0,240,180.0,0
2,2,20,10,0.0,10.0,0,45.0,0.0,0,45.0,0,30,40.0,1265.0,240,225.0,0
3,3,20,10,0.0,0.0,0,15.0,0.0,0,15.0,0,40,40.0,1280.0,240,240.0,0
4,4,20,10,40.0,30.0,0,1145.0,800.0,240,105.0,0,50,80.0,2425.0,480,345.0,0
5,5,20,10,0.0,20.0,0,75.0,0.0,0,75.0,0,60,80.0,2500.0,480,420.0,0
6,6,20,10,0.0,10.0,0,45.0,0.0,0,45.0,0,70,80.0,2545.0,480,465.0,0
7,7,20,10,0.0,0.0,0,15.0,0.0,0,15.0,0,80,80.0,2560.0,480,480.0,0
8,8,20,10,40.0,30.0,0,1145.0,800.0,240,105.0,0,90,120.0,3705.0,720,585.0,0
9,9,20,10,0.0,20.0,0,75.0,0.0,0,75.0,0,100,120.0,3780.0,720,660.0,0


In [17]:
### 用自定义函数计算另一个 Q_list 的成本 ###

Q_list_2 = [30.0, 0, 0, 0, 30.0, 0, 0, 0, 30.0, 0, 0, 0,30.0, 0, 0, 0, 30.0, 0, 0, 0, 30.0, 0, 0, 0, 30.0, 0, 0, 0, 30.0, 0, 0, 0, 30.0, 0, 0, 0, 30.0, 0, 0, 0, 50.0, 0, 0, 0, 50.0, 0, 0, 0, 50.0, 0, 0, 0, 50.0, 0, 0, 0, 50.0, 0, 0, 0]
df = inventory_cost_df(days, P_list, D_list, Q_list_2)
df[['累计需求量', '累计订货量', '累计总成本', '累计运输成本', '累计库存成本', '累计缺货成本']].iloc[-1]


累计需求量       600.0
累计订货量       550.0
累计总成本     42210.0
累计运输成本     3600.0
累计库存成本      210.0
累计缺货成本    27400.0
Name: 59, dtype: float64

# 任务二：优化求解总成本最低策略

## 2.1 用编程枚举方法，寻找 EOQ

### 2.1.1  n_k_pairs

In [18]:
### n_k_pairs：分 n 批，每批订购 k 个，共订购 n * k >= quantity, 可能的整数对集合 ###
import math

def n_k_pairs(quantity, days):
    n_k_pairs = []
    for n in range(1, days+1):
        # n 批需要订购的个数 k
        k = math.ceil(quantity / n) 
        n_k_pairs.append((n, k))
    return n_k_pairs

print(n_k_pairs(600, 60))

[(1, 600), (2, 300), (3, 200), (4, 150), (5, 120), (6, 100), (7, 86), (8, 75), (9, 67), (10, 60), (11, 55), (12, 50), (13, 47), (14, 43), (15, 40), (16, 38), (17, 36), (18, 34), (19, 32), (20, 30), (21, 29), (22, 28), (23, 27), (24, 25), (25, 24), (26, 24), (27, 23), (28, 22), (29, 21), (30, 20), (31, 20), (32, 19), (33, 19), (34, 18), (35, 18), (36, 17), (37, 17), (38, 16), (39, 16), (40, 15), (41, 15), (42, 15), (43, 14), (44, 14), (45, 14), (46, 14), (47, 13), (48, 13), (49, 13), (50, 12), (51, 12), (52, 12), (53, 12), (54, 12), (55, 11), (56, 11), (57, 11), (58, 11), (59, 11), (60, 10)]


### 2.1.2  nk_Q_lists

In [19]:
### nk_Q_lists：给定天数、总量，等量、整批、等间隔订购的可选策略集合 ###


# 给定 quantity，求 (n, k), 并得到所有可行的 Q_list
def nk_Q_lists(quantity, days):
    
    nk_Q_lists = []
    
    # 遍历 n, k pairs
    for n, k in n_k_pairs(quantity, days):
        
        Q_list = []
        
        # 遍历每日
        for day_i in range(days):
            # 已达到总数则后续全为 0
            if sum(Q_list) >= quantity:
                Q_list.append(0)
                continue 
            # 要求首日订购
            if day_i == 0:
                Q_list.append(k)
                continue  
            # 如果到了订货周期，则订购（如果今天再不订购，则截至今日结束进度落后）
            if sum(Q_list)/quantity < (day_i+1)/days:
                # 本次订购 k，但不超过 quantity
                if sum(Q_list) + k > quantity:
                    Q_list.append(quantity - sum(Q_list))
                else:
                    Q_list.append(k)
            else:
                Q_list.append(0)  
                
        # add to nk_Q_lists
        if Q_list not in nk_Q_lists:
            nk_Q_lists.append(Q_list)
        
    # return
    return nk_Q_lists

In [20]:
# 本例中 n, k 法可选 Q_list
for i in nk_Q_lists(600, 60):
    print(i)

[600, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[300, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 300, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[200, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 200, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 200, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[150, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 150, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 150, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 150, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[120, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 120, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 120, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 120, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 120, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### 2.1.3  solution_nk

In [21]:
### 遍历 n, k Q_list，寻找成本最低的策略 ###

def solution_nk(days, P_list, D_list):

    # lowest_record
    lowest_record = 9999999999
    quantity = sum(D_list)
    
    #print(n_k_pairs(quantity, days))
    
    # all possible stategies
    for Q_list in nk_Q_lists(quantity, days):
        # total cost for Q_list.
        cost = inventory_cost_df(days, P_list, D_list, Q_list, fast=True)['累计总成本'].iloc[days-1]
        # update lowest_record
        if cost <= lowest_record:
            lowest_record = cost
            best_Q_list = Q_list
            best_df = inventory_cost_df(days, P_list, D_list, best_Q_list)
    # return
    return best_df


In [22]:
### 验证EOQ ###

days = 60
P0 = [20 for i in range(days)]
D0 = [10 for i in range(days)]

solution_nk(days, P0, D0)


,日期,价格,需求量,订货量,库存量,缺货量,总成本,购进货物价值,运输成本,库存成本,缺货成本,累计需求量,累计订货量,累计总成本,累计运输成本,累计库存成本,累计缺货成本
0,0,20,10,40,30,0,1145.0,800,240,105.0,0,10,40,1145.0,240,105.0,0
1,1,20,10,0,20,0,75.0,0,0,75.0,0,20,40,1220.0,240,180.0,0
2,2,20,10,0,10,0,45.0,0,0,45.0,0,30,40,1265.0,240,225.0,0
3,3,20,10,0,0,0,15.0,0,0,15.0,0,40,40,1280.0,240,240.0,0
4,4,20,10,40,30,0,1145.0,800,240,105.0,0,50,80,2425.0,480,345.0,0
5,5,20,10,0,20,0,75.0,0,0,75.0,0,60,80,2500.0,480,420.0,0
6,6,20,10,0,10,0,45.0,0,0,45.0,0,70,80,2545.0,480,465.0,0
7,7,20,10,0,0,0,15.0,0,0,15.0,0,80,80,2560.0,480,480.0,0
8,8,20,10,40,30,0,1145.0,800,240,105.0,0,90,120,3705.0,720,585.0,0
9,9,20,10,0,20,0,75.0,0,0,75.0,0,100,120,3780.0,720,660.0,0


## 2.2 使用循环自定义一个策略

### 2.2.1 新场景：P 和 D 为变量

In [23]:
days=60
D1 = [10, 8, 6, 9, 4, 0, 3, 0, 0, 0, 5, 6, 14, 0, 1, 2, 3, 3, 6, 1, 4, 8, 15, 7, 9, 6, 9, 4, 8, 11, 3, 19, 18, 23, 21, 20, 15, 15, 17, 21, 10, 10, 18, 17, 15, 17, 12, 15, 12, 16, 18, 15, 12, 12, 10, 13, 13, 12, 10, 9]
P1 = [20, 19, 21, 20, 19, 21, 21, 19, 20, 18, 21, 21, 21, 18, 20, 20, 20, 20, 21, 17, 20, 19, 20, 22, 22, 20, 21, 18, 20, 21, 20, 20, 21, 21, 22, 20, 20, 19, 21, 22, 20, 19, 20, 18, 20, 19, 20, 20, 21, 20, 21, 19, 17, 20, 19, 20, 20, 20, 21, 22]


In [24]:
### 此时 n, k 法的结果（不理想）###

solution_nk(days, P1, D1)


,日期,价格,需求量,订货量,库存量,缺货量,总成本,购进货物价值,运输成本,库存成本,缺货成本,累计需求量,累计订货量,累计总成本,累计运输成本,累计库存成本,累计缺货成本
0,0,20,10,40,30,0,1145.0,800,240,105.0,0,10,40,1145.0,240,105.0,0
1,1,19,8,0,22,0,78.0,0,0,78.0,0,18,40,1223.0,240,183.0,0
2,2,21,6,0,16,0,57.0,0,0,57.0,0,24,40,1280.0,240,240.0,0
3,3,20,9,0,7,0,34.5,0,0,34.5,0,33,40,1314.5,240,274.5,0
4,4,19,4,40,43,0,1135.0,760,240,135.0,0,37,80,2449.5,480,409.5,0
5,5,21,0,0,43,0,129.0,0,0,129.0,0,37,80,2578.5,480,538.5,0
6,6,21,3,0,40,0,124.5,0,0,124.5,0,40,80,2703.0,480,663.0,0
7,7,19,0,0,40,0,120.0,0,0,120.0,0,40,80,2823.0,480,783.0,0
8,8,20,0,40,80,0,1280.0,800,240,240.0,0,40,120,4103.0,720,1023.0,0
9,9,18,0,0,80,0,240.0,0,0,240.0,0,40,120,4343.0,720,1263.0,0


### 2.2.2 自定义策略：solution_min_batchcost

In [37]:
### solution_min_batchcost: 使某一次订货量达到单次单位成本最低  ###

def solution_min_batchcost(days, P_list, D_list):

    # 空列表
    Q_list_selected = [0 for i in range(days)]
    Stock_list = [None for i in range(days)]
    
    # 逐日处理
    for day_i in range(days):

        # 如果昨日已经购进过，不再购进
        if Q_list_selected[day_i-1] != 0:
            best_Q = 0
        else:
            # reset record
            lowest_record = 9999999

            ### 进货目标为：使本次进货的单位成本最低 ###

            # 仍需购进的总数：remain_num, 期初库存：begin_stock
            if day_i == 0:
                remain_num = sum(D_list)
                begin_stock = 0
            else:
                remain_num = sum(D_list[day_i:])
                begin_stock = Stock_list[day_i-1]

            ### 假设当日订购 try_num 个，后续都是 0，判断单位成本（try_num: 1～200）###
            for try_num in range(1, min(remain_num + 1, 201)):
                # Planned_stock_list
                Planned_stock_list = [None for i in range(days)]
                if day_i == 0:
                    Planned_Q_list = [0 for i in range(0, days)]
                    Planned_Q_list[day_i] = try_num
                else:
                    Planned_Q_list = Q_list_selected[0:day_i] + [0 for i in range(day_i, days)]
                    Planned_Q_list[day_i] = try_num
                # cycle_end
                for cycle_end in range(day_i, days):
                    # cycle_end 这一天的日末库存
                    if day_i == 0 or cycle_end == day_i:
                        Planned_stock_list[cycle_end] = try_num - D_list[cycle_end]
                    else:
                        Planned_stock_list[cycle_end] = Planned_stock_list[cycle_end-1] - D_list[cycle_end]
                    # 如果库存 <= 0，则该周期结束，得到 cycle_end 的值
                    if Planned_stock_list[cycle_end] <= 0:
                        break

                ### 本周期的总成本 ###
                # 起始数：day_i-1 的总成本
                if day_i == 0:
                    cost_begin = 0
                else:
                    cost_begin = inventory_cost_df(days, P_list, D_list, Planned_Q_list, fast=True)['累计总成本'].iloc[day_i-1]
                # 截止数：cycle_end 的总成本
                cost_end = inventory_cost_df(days, P_list, D_list, Planned_Q_list, fast=True)['累计总成本'].iloc[cycle_end]
                # 订购 try_num 个货物的 unit_cost
                unit_cost_this_cycle = (cost_end - cost_begin) / try_num  
                # 寻找新的 lowest_record
                if unit_cost_this_cycle < lowest_record:
                    lowest_record = unit_cost_this_cycle
                    best_Q = try_num

            # 如果不进货，则需要考虑由于今日不进货所导致的缺货成本 (D_list[day_i] - begin_stock) * IC
            target_unit_cost = 18800 / sum(D_list)
            inaction_unit_cost = (D_list[day_i] - begin_stock) * IC / 1
            # unit cost 过高，则选择不进货：既高于设定成本上限，又高于本日不进货的假设成本
            if lowest_record > target_unit_cost and lowest_record > inaction_unit_cost:
                best_Q = 0

        # 本日 Q 确定完毕，记录到 Q_list_selected
        Q_list_selected[day_i] = best_Q
        # 更新实际历史库存信息
        if day_i == 0:
            Stock_list[day_i] = Q_list_selected[0] - D_list[day_i]
        else:
            Stock_list[day_i] = Stock_list[day_i-1] + Q_list_selected[day_i] - D_list[day_i]

    # cost of Q_list_selected
    best_df = inventory_cost_df(days, P_list, D_list, Q_list_selected)
    return best_df
    

In [38]:
# solution_min_batchcost result
our_Q_list = solution_min_batchcost(days, P1, D1)['订货量']
inventory_cost_df(days, P1, D1, our_Q_list)


,日期,价格,需求量,订货量,库存量,缺货量,总成本,购进货物价值,运输成本,库存成本,缺货成本,累计需求量,累计订货量,累计总成本,累计运输成本,累计库存成本,累计缺货成本
0,0,20,10,10,0,0,455.0,200,240,15.0,0,10,10,455.0,240,15.0,0
1,1,19,8,0,-8,8,80.0,0,0,0.0,80,18,10,535.0,240,15.0,80
2,2,21,6,33,19,0,999.0,693,240,66.0,0,24,43,1534.0,480,81.0,80
3,3,20,9,0,10,0,43.5,0,0,43.5,0,33,43,1577.5,480,124.5,80
4,4,19,4,0,6,0,24.0,0,0,24.0,0,37,43,1601.5,480,148.5,80
5,5,21,0,0,6,0,18.0,0,0,18.0,0,37,43,1619.5,480,166.5,80
6,6,21,3,0,3,0,13.5,0,0,13.5,0,40,43,1633.0,480,180.0,80
7,7,19,0,0,3,0,9.0,0,0,9.0,0,40,43,1642.0,480,189.0,80
8,8,20,0,0,3,0,9.0,0,0,9.0,0,40,43,1651.0,480,198.0,80
9,9,18,0,0,3,0,9.0,0,0,9.0,0,40,43,1660.0,480,207.0,80


In [39]:
# 刚刚的解
print([i for i in our_Q_list])


[10, 0, 33, 0, 0, 0, 0, 0, 0, 0, 0, 26, 0, 0, 0, 0, 0, 37, 0, 0, 0, 0, 0, 35, 0, 0, 0, 0, 41, 0, 0, 60, 0, 0, 56, 0, 0, 53, 0, 0, 0, 45, 0, 0, 44, 0, 0, 54, 0, 0, 0, 39, 0, 0, 48, 0, 0, 0, 19, 0]


## 2.3 使用优化算法

### 2.3.1 局部优化

In [28]:
### 使用优化算法 ###

from scipy import optimize


### 为了加速运算，重新定义总成本目标函数：inventory cost ###
def inventory_cost(days, P_list, D_list, Q_list, S=250, C=3.2, IC=12):
    # 准备空列表
    Stock_list = [None for i in range(days)]
    Cost_list = [None for i in range(days)]
    ### 按日处理 ###
    for day_i in range(days):
        # 昨日库存：
        if day_i == 0:
            l_stock = 0
        else:
            l_stock = Stock_list[day_i-1]
        # 今日价格：
        p = P_list[day_i]
        # 今日订货量：
        q = Q_list[day_i]
        # 今日需求量：
        d = D_list[day_i]
        # 更新今日库存 = 昨日库存 + 今日购进 - 今日使用量
        stock = l_stock + q - d # 今日结束时的库存（负数表示缺货）
        Stock_list[day_i] = stock # 记录
        # 今日缺货量：
        if Stock_list[day_i] < 0:
            ins = abs(Stock_list[day_i]) # 缺货量（正数）
        else:
            ins = 0
        ### 今日成本计算 ###
        # 购进产品总价值
        cost_pq = q * p 
        # 运输成本 ，如果进货（q > 0），则付出运输成本 S，否则为 0
        if q > 0:
            cost_ship = S 
        else:
            cost_ship = 0
        # 存储成本，假设日内需求均匀发生，带来半数的等效库存成本
        cost_stock = C * (max(stock, 0) + min(d, max(q + l_stock, 0))/2) 
        # 缺货成本 = IC * 缺货量（正数）
        cost_ins = IC * ins 
        # 总成本
        cost = cost_pq + cost_ship + cost_stock + cost_ins 
        # 修改相应 list 中该日的数值
        Cost_list[day_i] = cost
        
    return sum(Cost_list)

In [29]:
### 原方案结果+局部优化 ###
days=60
def min_batchcost_optimized(days, P_list, D_list):

    # start with solution_min_batchcost solution
    our_Q_list = solution_min_batchcost(days, P_list, D_list)['订货量']
        
    # optimize.minimize
    func = lambda x: inventory_cost(days, P_list, D_list, x)
    # x0 = [100] * 60 # 起始点
    x0 = our_Q_list # 起始点
    lw = [0] * days
    up = [200] * days
    our_Q_list_new = [int(i) for i in optimize.minimize(func, x0, bounds=list(zip(lw, up)))['x']]

    result_df = inventory_cost_df(days, P_list, D_list, our_Q_list_new)
    
    return result_df

min_batchcost_optimized(days, P1, D1)

,日期,价格,需求量,订货量,库存量,缺货量,总成本,购进货物价值,运输成本,库存成本,缺货成本,累计需求量,累计订货量,累计总成本,累计运输成本,累计库存成本,累计缺货成本
0,0,20,10,18,8,0,639.0,360,240,39.0,0,10,18,639.0,240,39.0,0
1,1,19,8,0,0,0,12.0,0,0,12.0,0,18,18,651.0,240,51.0,0
2,2,21,6,21,15,0,735.0,441,240,54.0,0,24,39,1386.0,480,105.0,0
3,3,20,9,0,6,0,31.5,0,0,31.5,0,33,39,1417.5,480,136.5,0
4,4,19,4,0,2,0,12.0,0,0,12.0,0,37,39,1429.5,480,148.5,0
5,5,21,0,0,2,0,6.0,0,0,6.0,0,37,39,1435.5,480,154.5,0
6,6,21,3,0,-1,1,13.0,0,0,3.0,10,40,39,1448.5,480,157.5,10
7,7,19,0,0,-1,1,10.0,0,0,0.0,10,40,39,1458.5,480,157.5,20
8,8,20,0,0,-1,1,10.0,0,0,0.0,10,40,39,1468.5,480,157.5,30
9,9,18,0,0,-1,1,10.0,0,0,0.0,10,40,39,1478.5,480,157.5,40


### 2.3.2 模拟退火 (Dual Annealing optimization)

In [45]:
### 模拟退火全局优化 ###

def min_batchcost_dual_annealing(days, P_list, D_list, startwith='min_batchcost'):

    if startwith=='min_batchcost':
        # start with solution_min_batchcost solution
        our_Q_list = solution_min_batchcost(days, P_list, D_list)['订货量']
        # optimize.dual_annealing
        func = lambda x: inventory_cost(days, P_list, D_list, x)
        x0 = our_Q_list # 起始点
        lw = [0] * days
        up = [200] * days
    else:
        # optimize.dual_annealing
        func = lambda x: inventory_cost(days, P_list, D_list, x)
        x0 = [100] * days # 起始点
        lw = [0] * days
        up = [200] * days

     #退火 1 次
    x_solution = optimize.dual_annealing(func, list(zip(lw, up)), maxiter=1000, x0=x0)['x']
    solution_Q_list = [int(i) for i in x_solution]
    best_solution = inventory_cost_df(days, P_list, D_list, solution_Q_list)

    # # 退火 3 次
    best_cost = 18900
    for i in range(3):
        print(f'now round {i}:')    
        # 优化求解
        x_solution = optimize.dual_annealing(func, list(zip(lw, up)), maxiter=1000, x0=x0)['x']
        solution_Q_list = [int(i) for i in x_solution]
    # 记录更优结果
        if inventory_cost(days, P1, D1, solution_Q_list) < best_cost:
            best_cost = inventory_cost(days, P_list, D_list, solution_Q_list)
            best_solution = inventory_cost_df(days, P_list, D_list, solution_Q_list)
            x0 = solution_Q_list
            print(f'new record: {best_cost}')
    
    return best_solution

min_batchcost_dual_annealing(days, P1, D1)

now round 0:
now round 1:
now round 2:


,日期,价格,需求量,订货量,库存量,缺货量,总成本,购进货物价值,运输成本,库存成本,缺货成本,累计需求量,累计订货量,累计总成本,累计运输成本,累计库存成本,累计缺货成本
0,0,20,10,18,8,0,639.0,360,240,39.0,0,10,18,639.0,240,39.0,0
1,1,19,8,0,0,0,12.0,0,0,12.0,0,18,18,651.0,240,51.0,0
2,2,21,6,21,15,0,735.0,441,240,54.0,0,24,39,1386.0,480,105.0,0
3,3,20,9,0,6,0,31.5,0,0,31.5,0,33,39,1417.5,480,136.5,0
4,4,19,4,0,2,0,12.0,0,0,12.0,0,37,39,1429.5,480,148.5,0
5,5,21,0,0,2,0,6.0,0,0,6.0,0,37,39,1435.5,480,154.5,0
6,6,21,3,0,-1,1,13.0,0,0,3.0,10,40,39,1448.5,480,157.5,10
7,7,19,0,0,-1,1,10.0,0,0,0.0,10,40,39,1458.5,480,157.5,20
8,8,20,0,0,-1,1,10.0,0,0,0.0,10,40,39,1468.5,480,157.5,30
9,9,18,0,0,-1,1,10.0,0,0,0.0,10,40,39,1478.5,480,157.5,40


## 2.4 四种策略对比：哪种效果最好？

In [31]:
#-----------------------------------------------------------------------------------------------#
# 四种策略对比：哪种效果最好？
#-----------------------------------------------------------------------------------------------#
days = 60
quantity = 600
import datetime
# 随机生成 3 次 P 和 D 的分布（进行 3 次比赛）
for i in range(3):
    
    # round
    print(f'\n第 {i+1} 次比赛：')
    # rand P_list, D_list
    P_list = [20, 19, 21, 20, 19, 21, 21, 19, 20, 18, 21, 21, 21, 18, 20, 20, 20, 20, 21, 17, 20, 19, 20, 22, 22, 20, 21, 18, 20, 21, 20, 20, 21, 21, 22, 20, 20, 19, 21, 22, 20, 19, 20, 18, 20, 19, 20, 20, 21, 20, 21, 19, 17, 20, 19, 20, 20, 20, 21, 22]
    D_list = [10, 8, 6, 9, 4, 0, 3, 0, 0, 0, 5, 6, 14, 0, 1, 2, 3, 3, 6, 1, 4, 8, 15, 7, 9, 6, 9, 4, 8, 11, 3, 19, 18, 23, 21, 20, 15, 15, 17, 21, 10, 10, 18, 17, 15, 17, 12, 15, 12, 16, 18, 15, 12, 12, 10, 13, 13, 12, 10, 9]
    
    # EOQ
    start_time = datetime.datetime.now()
    EOQ_cost = inventory_cost_df(days, P_list, D_list, Q_list_EOQ)['累计总成本'].iloc[days-1]
    end_time = datetime.datetime.now()
    ms = (end_time - start_time).microseconds
    sec = (end_time - start_time).seconds
    delta_time = sec + ms / 1000000
    print(f'[0] EOQ: cost = {EOQ_cost}, 计算耗时 {delta_time:.3f} seconds.')
    
    # solution_nk
    start_time = datetime.datetime.now()
    EOQ_cost = solution_nk(days, P_list, D_list)['累计总成本'].iloc[days-1]
    end_time = datetime.datetime.now()
    ms = (end_time - start_time).microseconds
    sec = (end_time - start_time).seconds
    delta_time = sec + ms / 1000000
    print(f'[1] solution_nk: cost = {EOQ_cost}, 计算耗时 {delta_time:.3f} seconds.')
    
    # solution_min_batchcost
    start_time = datetime.datetime.now()
    solution1_cost = solution_min_batchcost(days, P_list, D_list)['累计总成本'].iloc[days-1]
    end_time = datetime.datetime.now()
    ms = (end_time - start_time).microseconds
    sec = (end_time - start_time).seconds
    delta_time = sec + ms / 1000000
    print(f'[2] solution_min_batchcost: cost = {solution1_cost}, 计算耗时 {delta_time:.3f} seconds.')
    
    # min_batchcost_optimized
    start_time = datetime.datetime.now()
    solution2_cost = min_batchcost_optimized(days, P_list, D_list)['累计总成本'].iloc[days-1]
    end_time = datetime.datetime.now()
    ms = (end_time - start_time).microseconds
    sec = (end_time - start_time).seconds
    delta_time = sec + ms / 1000000
    print(f'[3] min_batchcost_optimized: cost = {solution2_cost}, 计算耗时 {delta_time:.3f} seconds.')
    
    # min_batchcost_dual_annealing
    start_time = datetime.datetime.now()
    solution3_cost = min_batchcost_dual_annealing(days, P_list, D_list)['累计总成本'].iloc[days-1]
    end_time = datetime.datetime.now()
    ms = (end_time - start_time).microseconds
    sec = (end_time - start_time).seconds
    delta_time = sec + ms / 1000000
    print(f'[4] min_batchcost_dual_annealing = {solution3_cost}, 计算耗时 {delta_time:.3f} seconds.')




第 1 次比赛：
[0] EOQ: cost = 31105.0, 计算耗时 0.004 seconds.
[1] solution_nk: cost = 31105.0, 计算耗时 0.083 seconds.
[2] solution_min_batchcost: cost = 19741.0, 计算耗时 19.103 seconds.
[3] min_batchcost_optimized: cost = 18880.5, 计算耗时 24.321 seconds.
now round 0:
now round 1:
now round 2:
[4] min_batchcost_dual_annealing = 18872.0, 计算耗时 215.188 seconds.

第 2 次比赛：
[0] EOQ: cost = 31105.0, 计算耗时 0.010 seconds.
[1] solution_nk: cost = 31105.0, 计算耗时 0.120 seconds.
[2] solution_min_batchcost: cost = 19741.0, 计算耗时 19.647 seconds.
[3] min_batchcost_optimized: cost = 18880.5, 计算耗时 25.155 seconds.
now round 0:
now round 1:
now round 2:
[4] min_batchcost_dual_annealing = 18880.5, 计算耗时 206.499 seconds.

第 3 次比赛：
[0] EOQ: cost = 31105.0, 计算耗时 0.010 seconds.
[1] solution_nk: cost = 31105.0, 计算耗时 0.149 seconds.
[2] solution_min_batchcost: cost = 19741.0, 计算耗时 20.546 seconds.
[3] min_batchcost_optimized: cost = 18880.5, 计算耗时 24.198 seconds.
now round 0:
now round 1:
now round 2:
[4] min_batchcost_dual_annealing =